# Optimization Model - Hybrid Energy Generation & Storage: 1 Battery-1 Genset.
=========================================================================================================================
$ 
\\
$

$ 
\\
$

### Importing Libraries.

In [1]:
import sys
import numpy as np
from pulp import *
from src.functions.load_window import lwd
from src.functions.add_dict import merge_dict

### Python environment.

In [2]:

print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=10, micro=9, releaselevel='final', serial=0)


### Introducing problem Parameters.

In [3]:
Q_max = 250                                                    # maximal capacity stored on battery
Q_0 = 0.2*Q_max                                                # inital capacity stored on battery
P_A_max = 800                                                  # max output from gen set A
dt = 1                                                         # time step in simulation dt=1min
t_max = 60                                                     # time span for simulation = t_max hours
t = np.atleast_2d(np.arange(0,t_max,dt)).T.conj()
n = len(t)                                                     # number of time steps
m = 1                                                          # number of Gensets used on the ship 
a_j = 1.5                                                      # slope for the linear model of genset j consumption 
fc_j_offset = 500                                              # genset j fuel consumption when no power gnerated (p=0) in g/h 
y_k_j = 1                                                      # genset j pointer at time step k (if genset j is used then = 1 otherwise = 0)
y_k_j_1 = 0                                                    # genset j pointer at time step k-1 (if genset j is used then = 1 otherwise = 0)
K_j_start = dict.fromkeys([x for x in range(60)],0)
K_j_start[0]=30                                                # additional fuel consumption when starting genset j (kg)




L = lwd (200,200,10,
         400,400,10,
         600,600,10,
         600,200,30,
         0,10,20,30
         )                                                      # Pre-defined load porfile requirements framed in 60 time steps

### Creating the problem variable. 

In [4]:
Optim = LpProblem('Energy_Opt',LpMinimize)

### Setting-up decision variables.

In [5]:
V_steps = [x for x in range (60)]

Q_bat = LpVariable.dicts("Q_bat", V_steps, lowBound=0.2*Q_max, upBound=Q_max, cat = LpContinuous)  # Battery charge at time step k.
P_bat = LpVariable.dicts("P_bat", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpContinuous) # Power generated by the battery.
P_A = LpVariable.dicts("P_A", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpInteger)  # Power generated by the Genset j.
P_A_load = LpVariable.dicts("P_A_load", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpInteger) # Power required by the load at time step k.
Z_k_j =  LpVariable.dicts("Z_k_j", V_steps, lowBound=0, upBound=1, cat = LpBinary) # Aditional cost fuel oil consumption when starting Genset j.
SFOC_A = LpVariable.dicts("SFOC_A", V_steps, lowBound=0, upBound=None, cat = LpInteger) # Specific fuel oil consumption.
P_A_bat =  LpVariable.dicts("P_A_bat", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpContinuous) 
Y_to_bat = LpVariable.dicts("Y_to_bat", V_steps, lowBound=0, upBound=1, cat = LpBinary) # Genset selecter to charge the battery at time step k. 
Y_from_bat = LpVariable.dicts("Y_to_bat", V_steps, lowBound=0, upBound=1, cat = LpBinary) # Battery selecter to transfert to the Genset j st time step k.

### Setting-up the objective function.

In [6]:
for k in V_steps:
  for j in range(1,m+1):    
    FC_k_j = {k: (P_A[k]*a_j + fc_j_offset * y_k_j)  for k in V_steps}   # linear model for fuel consumption of genset j at time step k
    FC = (sum(FC_k_j[k] * dt/1000 * k for k in V_steps ) ) # sum of the fuel oil comsumption for all gensets over all k steps.
    L_added_cost = sum(Z_k_j[j] * j for j in range(1,m+1)) * sum( K_j_start[k] * k for k in V_steps ) # sum of all of the additional costs including starting costs. 
    Optim += lpSum( FC + L_added_cost ), "objective function minimization fuel oil consumption" 


c:\Users\ahmmab2\.conda\envs\HYH_env\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


### Setting-up problem Constraints.

In [7]:

for k in V_steps: 

  Optim +=  lpSum (merge_dict (P_A_load, P_bat)) == L  # Meet Power Requirement at each time step k
  Optim +=  P_A_load[k] + P_A_bat[k] == P_A[k] #Power Split

  P_A_A = {k: (P_A[k]*(-67/P_A_max) + 260) for k in P_A} 
  Optim += SFOC_A [k] == P_A_A[k]  # Specific Fuel oil consumption


  Q_k =  {k: (P_A_bat[k] - P_bat[k])*dt for k in V_steps} 
  Optim += Q_bat[0] == Q_0 #nitial Charge balance
  Optim += Q_bat[k] + Q_k[k] == Q_bat[k] #Charge balance at time step k" 

  # Genset logical constraints. 
  Optim += P_A[k]  <= (0.9 * P_A_max * y_k_j) # upper bound logical constraint
  Optim += P_A[k]  >=  (0.2 * P_A_max * y_k_j) # lower bound logical constraint


  for j in range(1, m): 
    Optim += lpSum( (Y_to_bat[j] + Y_from_bat[j])  <= 1), #make sure charge and transfer of battery are not runing at the same time"



### Solving the problem.

In [8]:
status = Optim.solve()

### Optimization result.

In [9]:
LpStatus[status]

'Infeasible'

### Printing constraints per each time step.

In [10]:
print(Optim.variables)

<bound method LpProblem.variables of Energy_Opt:
MINIMIZE
0.0015*P_A_1 + 0.015*P_A_10 + 0.0165*P_A_11 + 0.018000000000000002*P_A_12 + 0.0195*P_A_13 + 0.021*P_A_14 + 0.0225*P_A_15 + 0.024*P_A_16 + 0.025500000000000002*P_A_17 + 0.027*P_A_18 + 0.0285*P_A_19 + 0.003*P_A_2 + 0.03*P_A_20 + 0.0315*P_A_21 + 0.033*P_A_22 + 0.0345*P_A_23 + 0.036000000000000004*P_A_24 + 0.0375*P_A_25 + 0.039*P_A_26 + 0.0405*P_A_27 + 0.042*P_A_28 + 0.043500000000000004*P_A_29 + 0.0045000000000000005*P_A_3 + 0.045*P_A_30 + 0.0465*P_A_31 + 0.048*P_A_32 + 0.0495*P_A_33 + 0.051000000000000004*P_A_34 + 0.0525*P_A_35 + 0.054*P_A_36 + 0.0555*P_A_37 + 0.057*P_A_38 + 0.0585*P_A_39 + 0.006*P_A_4 + 0.06*P_A_40 + 0.0615*P_A_41 + 0.063*P_A_42 + 0.0645*P_A_43 + 0.066*P_A_44 + 0.0675*P_A_45 + 0.069*P_A_46 + 0.07050000000000001*P_A_47 + 0.07200000000000001*P_A_48 + 0.0735*P_A_49 + 0.0075*P_A_5 + 0.075*P_A_50 + 0.0765*P_A_51 + 0.078*P_A_52 + 0.0795*P_A_53 + 0.081*P_A_54 + 0.0825*P_A_55 + 0.084*P_A_56 + 0.0855*P_A_57 + 0.0870000000

In [11]:
for v in Optim.variables():
    print(v.name, "=", v.varValue)

P_A_0 = 720.0
P_A_1 = 720.0
P_A_10 = 720.0
P_A_11 = 720.0
P_A_12 = 320.0
P_A_13 = 320.0
P_A_14 = 320.0
P_A_15 = 320.0
P_A_16 = 320.0
P_A_17 = 320.0
P_A_18 = 320.0
P_A_19 = 320.0
P_A_2 = 720.0
P_A_20 = 320.0
P_A_21 = 320.0
P_A_22 = 320.0
P_A_23 = 320.0
P_A_24 = 320.0
P_A_25 = 320.0
P_A_26 = 320.0
P_A_27 = 320.0
P_A_28 = 320.0
P_A_29 = 320.0
P_A_3 = 720.0
P_A_30 = 320.0
P_A_31 = 320.0
P_A_32 = 320.0
P_A_33 = 320.0
P_A_34 = 320.0
P_A_35 = 320.0
P_A_36 = 320.0
P_A_37 = 320.0
P_A_38 = 320.0
P_A_39 = 320.0
P_A_4 = 720.0
P_A_40 = 320.0
P_A_41 = 320.0
P_A_42 = 320.0
P_A_43 = 320.0
P_A_44 = 320.0
P_A_45 = 320.0
P_A_46 = 320.0
P_A_47 = 320.0
P_A_48 = 320.0
P_A_49 = 320.0
P_A_5 = 720.0
P_A_50 = 320.0
P_A_51 = 320.0
P_A_52 = 320.0
P_A_53 = 320.0
P_A_54 = 320.0
P_A_55 = 320.0
P_A_56 = 320.0
P_A_57 = 320.0
P_A_58 = 320.0
P_A_59 = 320.0
P_A_6 = 720.0
P_A_7 = 720.0
P_A_8 = 720.0
P_A_9 = 720.0
P_A_bat_0 = 160.0
P_A_bat_1 = 160.0
P_A_bat_10 = 160.0
P_A_bat_11 = 160.0
P_A_bat_12 = 160.0
P_A_bat_13 = 160.

### Calculating the optimized fuel comsumption.

In [ ]:
print("Total fuel comsumption of the trip:", value(Optim.objective),'Kg')

Total fuel comsumption of the trip: 1774.1999999999998 Kg
